In [571]:
import pandas as pd
from sklearn.preprocessing import normalize
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from pathlib import Path

# Importing our survey data

In [572]:
file = Path('Resources/survey_results.xlsx')

results_df = pd.read_excel(file)
results_df

,ID,Start time,Completion time,Email,Name,What is your age range?,Gender,Race/Ethnicity (Select all that apply),Which US Geographic Region do you live in?,What is the name of your favorite TV Show?\n,How often do you eat vegetables?,Types of veggies you most like to eat (pick your top 5!),"What is your ""go to"" choice for veggies at home? (Pick your top three)","What is your ""go to"" choice for veggies at a restaurant (top three)",Preferred ways for cooking/preparing the choices you selected above,What alternative to vegetables do you use/eat to stay healthy?,"What are your top three favorite types of fruits you like to eat? (example: apples, bananas, oranges)"
0,1,2022-06-04 15:01:02,2022-06-04 15:05:26,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Firefly,1/week,Asparagus ;Corn;Carrots;Spinach;Broccoli;,Lettuce (any kind);Carrots;Spinach;,Asparagus;Corn;Broccoli;,Grilled;Sauteed;Baked;,NaN,"Cherries, blueberries, pineapple"
1,2,2022-06-06 19:20:23,2022-06-06 19:22:18,anonymous,NaN,20-29,Female,Asian;,Midwest,Vincenzo,4-5/week,Broccoli;Cauliflower;Corn;Green Beans;Peas;Spi...,Broccoli;Corn;Peas;,Broccoli;Cauliflower;Corn;,Fried/Air-fried;Grilled;Sauteed;Microwaved;,NaN,Mangoes cherries oranges
2,3,2022-06-06 19:19:21,2022-06-06 19:22:26,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Ted Lasso,Daily 6-7 week,Broccoli;Carrots;Green Beans;Spinach;Squash;,Broccoli;Carrots;Squash;,Lettuce (any kind);Broccoli;Spinach;,Steamed;Baked;,NaN,"bananas, strawberries, mangos"
3,4,2022-06-06 19:20:24,2022-06-06 19:25:18,anonymous,NaN,20-29,Female,Black or African American;,Midwest,Succession,2-3/week,Broccoli;Carrots;Corn;Green Beans;Cauliflower;,Broccoli;Carrots;Cauliflower;,Broccoli;Carrots;Green Beans;,Baked;Fried/Air-fried;Steamed;,NaN,"Bananas, strawberries, oranges"
4,5,2022-06-06 19:28:42,2022-06-06 19:35:54,anonymous,NaN,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Critical Role,2-3/week,Carrots;Spinach;Squash;Corn;Cauliflower;,Spinach;Carrots;Broccoli;,Spinach;Carrots;Squash;,Baked;Sauteed;,NaN,"Peaches, Oranges, Mangoes"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,157,2022-06-07 15:11:42,2022-06-07 15:13:43,anonymous,NaN,20-29,Female,Asian;,Midwest,90 Day Fiance,Daily 6-7 week,Lettuce (any kind);Spinach;Corn;asparagus;Carr...,Lettuce (any kind);Spinach;Carrots;,brussel sprouts;Spinach;Corn;,Fried/Air-fried;Sauteed;,NaN,"blueberries, strawberries, grapes"
157,158,2022-06-07 15:47:11,2022-06-07 15:51:49,anonymous,NaN,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Gilmore girls,Daily 6-7 week,Spinach;Lettuce (any kind);Carrots;Broccoli;Sq...,Broccoli;Spinach;Lettuce (any kind);,Squash;Spinach;Cauliflower;,Baked;Sauteed;Steamed;,NaN,"Bananas, apples, grapes"
158,159,2022-06-07 15:59:49,2022-06-07 16:03:12,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Chicago Fire,Daily 6-7 week,Lettuce (any kind);Corn;Cauliflower;Carrots;Cu...,Peas;Broccoli;Cauliflower;,Asparagus;,Fried/Air-fried;Grilled;Steamed;,NaN,"Bananas, grapes, apples"
159,160,2022-06-07 16:37:15,2022-06-07 16:39:20,anonymous,NaN,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Bones,2-3/week,Carrots;Corn;Peas;Spinach;Green Beans;,Carrots;Corn;Peas;,Carrots;Green Beans;Spinach;,Steamed;,NaN,"Apples, oranges, pineapple"


# Preprocessing

In [573]:
# renaming columns for ease of understanding
survey_df = results_df.rename(columns={
    "ID":"id",
    "Start time": "start_time",
    "Completion time": "end_time",
    "Email": "email",
    "Name": "name",	
    "What is your age range?": "age",
    "Gender":"gender",	
    "Race/Ethnicity (Select all that apply)": "race",	
    "Which US Geographic Region do you live in?": "region",	
    "What is the name of your favorite TV Show?\n": "tv_show",
    "How often do you eat vegetables?": "frequency",
    "Types of veggies you most like to eat (pick your top 5!)": "to_eat",
    'What is your "go to" choice for veggies at home? (Pick your top three)': "at_home",
    'What is your "go to" choice for veggies at a restaurant (top three)': "at_restaurant",	
    "Preferred ways for cooking/preparing the choices you selected above": "cook_prep",
    "What alternative to vegetables do you use/eat to stay healthy?": "alternatives",
    "What are your top three favorite types of fruits you like to eat? (example: apples, bananas, oranges)": "fruits"})
survey_df

,id,start_time,end_time,email,name,age,gender,race,region,tv_show,frequency,to_eat,at_home,at_restaurant,cook_prep,alternatives,fruits
0,1,2022-06-04 15:01:02,2022-06-04 15:05:26,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Firefly,1/week,Asparagus ;Corn;Carrots;Spinach;Broccoli;,Lettuce (any kind);Carrots;Spinach;,Asparagus;Corn;Broccoli;,Grilled;Sauteed;Baked;,NaN,"Cherries, blueberries, pineapple"
1,2,2022-06-06 19:20:23,2022-06-06 19:22:18,anonymous,NaN,20-29,Female,Asian;,Midwest,Vincenzo,4-5/week,Broccoli;Cauliflower;Corn;Green Beans;Peas;Spi...,Broccoli;Corn;Peas;,Broccoli;Cauliflower;Corn;,Fried/Air-fried;Grilled;Sauteed;Microwaved;,NaN,Mangoes cherries oranges
2,3,2022-06-06 19:19:21,2022-06-06 19:22:26,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Ted Lasso,Daily 6-7 week,Broccoli;Carrots;Green Beans;Spinach;Squash;,Broccoli;Carrots;Squash;,Lettuce (any kind);Broccoli;Spinach;,Steamed;Baked;,NaN,"bananas, strawberries, mangos"
3,4,2022-06-06 19:20:24,2022-06-06 19:25:18,anonymous,NaN,20-29,Female,Black or African American;,Midwest,Succession,2-3/week,Broccoli;Carrots;Corn;Green Beans;Cauliflower;,Broccoli;Carrots;Cauliflower;,Broccoli;Carrots;Green Beans;,Baked;Fried/Air-fried;Steamed;,NaN,"Bananas, strawberries, oranges"
4,5,2022-06-06 19:28:42,2022-06-06 19:35:54,anonymous,NaN,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Critical Role,2-3/week,Carrots;Spinach;Squash;Corn;Cauliflower;,Spinach;Carrots;Broccoli;,Spinach;Carrots;Squash;,Baked;Sauteed;,NaN,"Peaches, Oranges, Mangoes"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,157,2022-06-07 15:11:42,2022-06-07 15:13:43,anonymous,NaN,20-29,Female,Asian;,Midwest,90 Day Fiance,Daily 6-7 week,Lettuce (any kind);Spinach;Corn;asparagus;Carr...,Lettuce (any kind);Spinach;Carrots;,brussel sprouts;Spinach;Corn;,Fried/Air-fried;Sauteed;,NaN,"blueberries, strawberries, grapes"
157,158,2022-06-07 15:47:11,2022-06-07 15:51:49,anonymous,NaN,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Gilmore girls,Daily 6-7 week,Spinach;Lettuce (any kind);Carrots;Broccoli;Sq...,Broccoli;Spinach;Lettuce (any kind);,Squash;Spinach;Cauliflower;,Baked;Sauteed;Steamed;,NaN,"Bananas, apples, grapes"
158,159,2022-06-07 15:59:49,2022-06-07 16:03:12,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Chicago Fire,Daily 6-7 week,Lettuce (any kind);Corn;Cauliflower;Carrots;Cu...,Peas;Broccoli;Cauliflower;,Asparagus;,Fried/Air-fried;Grilled;Steamed;,NaN,"Bananas, grapes, apples"
159,160,2022-06-07 16:37:15,2022-06-07 16:39:20,anonymous,NaN,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Bones,2-3/week,Carrots;Corn;Peas;Spinach;Green Beans;,Carrots;Corn;Peas;,Carrots;Green Beans;Spinach;,Steamed;,NaN,"Apples, oranges, pineapple"


In [574]:
# this cell splits columns with multiple answers into their own columns, drops any extra answers beyond what was asked for, and names the new columns
# the result is five baby dataframes

drop_list_five = [0,1,2,3,4]
drop_list_three = [0,1,2]

# SPLIT COLUMNS WITH MULTIPLE SELECTIONS
to_eat_test = survey_df["to_eat"].str.split(';', expand = True)
at_home_test = survey_df["at_home"].str.split(';', expand = True)
at_restaurant_test = survey_df["at_restaurant"].str.split(';', expand = True)
cook_prep_test = survey_df["cook_prep"].str.split(';', expand = True)
fruits_test = survey_df["fruits"].str.split(', ', expand = True)

# drop any columns that come after our drop list
top_to_eat = to_eat_test.drop(to_eat_test.columns.difference(drop_list_five), axis = 1)
top_at_home = at_home_test.drop(at_home_test.columns.difference(drop_list_three), axis = 1)
top_at_restaurant = at_restaurant_test.drop(at_restaurant_test.columns.difference(drop_list_three), axis = 1)
top_cook_prep = cook_prep_test.drop(cook_prep_test.columns.difference(drop_list_three), axis = 1)
top_fruits = fruits_test.drop(fruits_test.columns.difference(drop_list_three), axis = 1)

# rename the new columns
top_to_eat = top_to_eat.rename(columns = {0:"to_eat_1", 1: "to_eat_2", 2: "to_eat_3", 3: "to_eat_4", 4: "to_eat_5"})
top_at_home = top_at_home.rename(columns = {0:"at_home_1", 1: "at_home_2", 2: "at_home_3"})
top_at_restaurant = top_at_restaurant.rename(columns = {0:"at_restaurant_1", 1: "at_restaurant_2", 2: "at_restaurant_3"})
top_cook_prep = top_cook_prep.rename(columns = {0:"cook_prep_1", 1: "cook_prep_2", 2: "cook_prep_3"})
top_fruits = top_fruits.rename(columns = {0:"fruits_1", 1: "fruits_2", 2: "fruits_3"})

In [575]:
# creating a list of new dataframes
df_list = [top_to_eat, top_at_home, top_at_restaurant,top_cook_prep, top_fruits]

# merging the five new dataframes into one new dataframe
df = df_list[0]
for df_ in df_list[1:]:
    df = df.merge(df_, left_index=True, right_index=True)

# adding this new dataframe back into survey_df 
for column in df:
    survey_df[column] = df[column]

print(survey_df.keys())
survey_df


Index(['id', 'start_time', 'end_time', 'email', 'name', 'age', 'gender',
       'race', 'region', 'tv_show', 'frequency', 'to_eat', 'at_home',
       'at_restaurant', 'cook_prep', 'alternatives', 'fruits', 'to_eat_1',
       'to_eat_2', 'to_eat_3', 'to_eat_4', 'to_eat_5', 'at_home_1',
       'at_home_2', 'at_home_3', 'at_restaurant_1', 'at_restaurant_2',
       'at_restaurant_3', 'cook_prep_1', 'cook_prep_2', 'cook_prep_3',
       'fruits_1', 'fruits_2', 'fruits_3'],
      dtype='object')


,id,start_time,end_time,email,name,age,gender,race,region,tv_show,...,at_home_3,at_restaurant_1,at_restaurant_2,at_restaurant_3,cook_prep_1,cook_prep_2,cook_prep_3,fruits_1,fruits_2,fruits_3
0,1,2022-06-04 15:01:02,2022-06-04 15:05:26,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Firefly,...,Spinach,Asparagus,Corn,Broccoli,Grilled,Sauteed,Baked,Cherries,blueberries,pineapple
1,2,2022-06-06 19:20:23,2022-06-06 19:22:18,anonymous,NaN,20-29,Female,Asian;,Midwest,Vincenzo,...,Peas,Broccoli,Cauliflower,Corn,Fried/Air-fried,Grilled,Sauteed,Mangoes cherries oranges,None,None
2,3,2022-06-06 19:19:21,2022-06-06 19:22:26,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Ted Lasso,...,Squash,Lettuce (any kind),Broccoli,Spinach,Steamed,Baked,,bananas,strawberries,mangos
3,4,2022-06-06 19:20:24,2022-06-06 19:25:18,anonymous,NaN,20-29,Female,Black or African American;,Midwest,Succession,...,Cauliflower,Broccoli,Carrots,Green Beans,Baked,Fried/Air-fried,Steamed,Bananas,strawberries,oranges
4,5,2022-06-06 19:28:42,2022-06-06 19:35:54,anonymous,NaN,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Critical Role,...,Broccoli,Spinach,Carrots,Squash,Baked,Sauteed,,Peaches,Oranges,Mangoes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,157,2022-06-07 15:11:42,2022-06-07 15:13:43,anonymous,NaN,20-29,Female,Asian;,Midwest,90 Day Fiance,...,Carrots,brussel sprouts,Spinach,Corn,Fried/Air-fried,Sauteed,,blueberries,strawberries,grapes
157,158,2022-06-07 15:47:11,2022-06-07 15:51:49,anonymous,NaN,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Gilmore girls,...,Lettuce (any kind),Squash,Spinach,Cauliflower,Baked,Sauteed,Steamed,Bananas,apples,grapes
158,159,2022-06-07 15:59:49,2022-06-07 16:03:12,anonymous,NaN,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Chicago Fire,...,Cauliflower,Asparagus,,None,Fried/Air-fried,Grilled,Steamed,Bananas,grapes,apples
159,160,2022-06-07 16:37:15,2022-06-07 16:39:20,anonymous,NaN,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Bones,...,Peas,Carrots,Green Beans,Spinach,Steamed,,None,Apples,oranges,pineapple


In [576]:
# dropping non-beneficial columns
survey_df.drop(['id', 'start_time', 'end_time', 'email', 'name', 'alternatives', 'to_eat', 'at_home', 'at_restaurant','cook_prep', 'fruits'], axis=1, inplace=True)
survey_df

,age,gender,race,region,tv_show,frequency,to_eat_1,to_eat_2,to_eat_3,to_eat_4,...,at_home_3,at_restaurant_1,at_restaurant_2,at_restaurant_3,cook_prep_1,cook_prep_2,cook_prep_3,fruits_1,fruits_2,fruits_3
0,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Firefly,1/week,Asparagus,Corn,Carrots,Spinach,...,Spinach,Asparagus,Corn,Broccoli,Grilled,Sauteed,Baked,Cherries,blueberries,pineapple
1,20-29,Female,Asian;,Midwest,Vincenzo,4-5/week,Broccoli,Cauliflower,Corn,Green Beans,...,Peas,Broccoli,Cauliflower,Corn,Fried/Air-fried,Grilled,Sauteed,Mangoes cherries oranges,None,None
2,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Ted Lasso,Daily 6-7 week,Broccoli,Carrots,Green Beans,Spinach,...,Squash,Lettuce (any kind),Broccoli,Spinach,Steamed,Baked,,bananas,strawberries,mangos
3,20-29,Female,Black or African American;,Midwest,Succession,2-3/week,Broccoli,Carrots,Corn,Green Beans,...,Cauliflower,Broccoli,Carrots,Green Beans,Baked,Fried/Air-fried,Steamed,Bananas,strawberries,oranges
4,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Critical Role,2-3/week,Carrots,Spinach,Squash,Corn,...,Broccoli,Spinach,Carrots,Squash,Baked,Sauteed,,Peaches,Oranges,Mangoes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,20-29,Female,Asian;,Midwest,90 Day Fiance,Daily 6-7 week,Lettuce (any kind),Spinach,Corn,asparagus,...,Carrots,brussel sprouts,Spinach,Corn,Fried/Air-fried,Sauteed,,blueberries,strawberries,grapes
157,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Gilmore girls,Daily 6-7 week,Spinach,Lettuce (any kind),Carrots,Broccoli,...,Lettuce (any kind),Squash,Spinach,Cauliflower,Baked,Sauteed,Steamed,Bananas,apples,grapes
158,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Chicago Fire,Daily 6-7 week,Lettuce (any kind),Corn,Cauliflower,Carrots,...,Cauliflower,Asparagus,,None,Fried/Air-fried,Grilled,Steamed,Bananas,grapes,apples
159,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Bones,2-3/week,Carrots,Corn,Peas,Spinach,...,Peas,Carrots,Green Beans,Spinach,Steamed,,None,Apples,oranges,pineapple


In [577]:
# determine the number of unique values in each column
survey_cat = survey_df.dtypes.index.tolist()
survey_df[survey_cat].nunique()

age                  7
gender               3
race                11
region               9
tv_show            126
frequency            5
to_eat_1            13
to_eat_2            12
to_eat_3            14
to_eat_4            23
to_eat_5            28
at_home_1           18
at_home_2           16
at_home_3           22
at_restaurant_1     22
at_restaurant_2     16
at_restaurant_3     25
cook_prep_1         10
cook_prep_2         10
cook_prep_3         10
fruits_1            62
fruits_2            61
fruits_3            60
dtype: int64

In [578]:
# look at fruits_1 value counts for binning
fruits_1_count = survey_df['fruits_1'].value_counts()
fruits_1_count

Apples                            21
Bananas                           12
Strawberries                      11
Banana                            10
Oranges                            7
                                  ..
Mangoes cherries oranges           1
Watermelon Strawberries Banana     1
orange pineapple banana            1
Graded                             1
Grapes water melon                 1
Name: fruits_1, Length: 62, dtype: int64

In [579]:
# bin all the fruits that have just 1 response together into "other"
fruits_1_to_replace = list(fruits_1_count[fruits_1_count == 1].index)

# replace in dataframe
for fruit in fruits_1_to_replace:
    survey_df['fruits_1'] = survey_df['fruits_1'].replace(fruit, "OTHER")

# check to make sure binning was successful
survey_df['fruits_1'].value_counts()

OTHER           42
Apples          21
Bananas         12
Strawberries    11
Banana          10
bananas          7
Blueberries      7
Oranges          7
Mango            7
Grapes           6
apples           5
Pineapple        4
Blackberries     3
Berries          3
Cherries         3
Mangoes          3
Raspberries      2
Watermelon       2
blueberries      2
Strawberry       2
Blueberry        2
Name: fruits_1, dtype: int64

In [580]:
# look at fruits_2 value counts for binning
fruits_2_count = survey_df['fruits_2'].value_counts()
fruits_2_count.head(30)

grapes                   15
bananas                  10
blueberries               9
strawberries              8
mango                     6
apples                    6
oranges                   5
raspberries               5
pineapple                 5
watermelon                4
banana                    4
Strawberries              3
peaches                   3
cantelope                 2
pears                     2
grapefruit                2
orange                    2
mangos                    2
Apples                    2
strawberry                2
Mango                     1
Papaya                    1
pear                      1
APPLES                    1
grapes,apples             1
tomatoes                  1
BANANAS                   1
honeydew (any) melons     1
pineapples                1
Oranges                   1
Name: fruits_2, dtype: int64

In [581]:
# bin all the fruits that have just 1 response together into "other"
fruits_2_to_replace = list(fruits_2_count[fruits_2_count == 1].index)

# replace in dataframe
for fruit in fruits_2_to_replace:
    survey_df['fruits_2'] = survey_df['fruits_2'].replace(fruit, "OTHER")

# check to make sure binning was successful
survey_df['fruits_2'].value_counts()

OTHER           41
grapes          15
bananas         10
blueberries      9
strawberries     8
apples           6
mango            6
pineapple        5
oranges          5
raspberries      5
banana           4
watermelon       4
peaches          3
Strawberries     3
pears            2
grapefruit       2
orange           2
mangos           2
cantelope        2
strawberry       2
Apples           2
Name: fruits_2, dtype: int64

In [582]:
# look at fruits_3 value counts for binning
fruits_3_count = survey_df['fruits_3'].value_counts()
fruits_3_count.head(30)

grapes           15
apples            9
oranges           8
bananas           5
kiwi              5
pineapple         5
blueberries       4
watermelon        4
banana            4
strawberries      4
peaches           3
strawberries      3
blackberries      2
Banana            2
grapes            2
blueberries       2
watermelon        2
oranges           2
Bananas           2
raspberry         2
strawberry        2
cherries          2
berries           2
Apples            2
melon             1
CANTELOPE         1
Clementines       1
Cherries          1
STRAWBERRIES      1
Grapes            1
Name: fruits_3, dtype: int64

In [583]:
# bin all the fruits that have just 1 response together into "other"
fruits_3_to_replace = list(fruits_3_count[fruits_3_count == 1].index)

# replace in dataframe
for fruit in fruits_3_to_replace:
    survey_df['fruits_3'] = survey_df['fruits_3'].replace(fruit, "OTHER")

# check to make sure binning was successful
survey_df['fruits_3'].value_counts()

OTHER            36
grapes           15
apples            9
oranges           8
kiwi              5
bananas           5
pineapple         5
banana            4
strawberries      4
blueberries       4
watermelon        4
strawberries      3
peaches           3
raspberry         2
grapes            2
Banana            2
blueberries       2
blackberries      2
watermelon        2
oranges           2
Bananas           2
strawberry        2
cherries          2
berries           2
Apples            2
Name: fruits_3, dtype: int64

In [584]:
survey_df

,age,gender,race,region,tv_show,frequency,to_eat_1,to_eat_2,to_eat_3,to_eat_4,...,at_home_3,at_restaurant_1,at_restaurant_2,at_restaurant_3,cook_prep_1,cook_prep_2,cook_prep_3,fruits_1,fruits_2,fruits_3
0,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Firefly,1/week,Asparagus,Corn,Carrots,Spinach,...,Spinach,Asparagus,Corn,Broccoli,Grilled,Sauteed,Baked,Cherries,blueberries,OTHER
1,20-29,Female,Asian;,Midwest,Vincenzo,4-5/week,Broccoli,Cauliflower,Corn,Green Beans,...,Peas,Broccoli,Cauliflower,Corn,Fried/Air-fried,Grilled,Sauteed,OTHER,None,None
2,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Ted Lasso,Daily 6-7 week,Broccoli,Carrots,Green Beans,Spinach,...,Squash,Lettuce (any kind),Broccoli,Spinach,Steamed,Baked,,bananas,strawberries,OTHER
3,20-29,Female,Black or African American;,Midwest,Succession,2-3/week,Broccoli,Carrots,Corn,Green Beans,...,Cauliflower,Broccoli,Carrots,Green Beans,Baked,Fried/Air-fried,Steamed,Bananas,strawberries,oranges
4,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Critical Role,2-3/week,Carrots,Spinach,Squash,Corn,...,Broccoli,Spinach,Carrots,Squash,Baked,Sauteed,,OTHER,OTHER,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,20-29,Female,Asian;,Midwest,90 Day Fiance,Daily 6-7 week,Lettuce (any kind),Spinach,Corn,asparagus,...,Carrots,brussel sprouts,Spinach,Corn,Fried/Air-fried,Sauteed,,blueberries,strawberries,grapes
157,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Gilmore girls,Daily 6-7 week,Spinach,Lettuce (any kind),Carrots,Broccoli,...,Lettuce (any kind),Squash,Spinach,Cauliflower,Baked,Sauteed,Steamed,Bananas,apples,grapes
158,40-49,Female,"Caucasian, non-Hispanic;",Midwest,Chicago Fire,Daily 6-7 week,Lettuce (any kind),Corn,Cauliflower,Carrots,...,Cauliflower,Asparagus,,None,Fried/Air-fried,Grilled,Steamed,Bananas,grapes,apples
159,30-39,Female,"Caucasian, non-Hispanic;",Midwest,Bones,2-3/week,Carrots,Corn,Peas,Spinach,...,Peas,Carrots,Green Beans,Spinach,Steamed,,None,Apples,oranges,pineapple


In [585]:
# re-look at the number of unique values in each column now that the fruits have been binned
survey_cat = survey_df.dtypes.index.tolist()
survey_df[survey_cat].nunique()

age                  7
gender               3
race                11
region               9
tv_show            126
frequency            5
to_eat_1            13
to_eat_2            12
to_eat_3            14
to_eat_4            23
to_eat_5            28
at_home_1           18
at_home_2           16
at_home_3           22
at_restaurant_1     22
at_restaurant_2     16
at_restaurant_3     25
cook_prep_1         10
cook_prep_2         10
cook_prep_3         10
fruits_1            21
fruits_2            21
fruits_3            25
dtype: int64

In [586]:
# make sure there are no null values
for column in survey_df.columns:
    print(f"Column {column} has {survey_df[column].isnull().sum()} null values")

Column age has 0 null values
Column gender has 0 null values
Column race has 0 null values
Column region has 0 null values
Column tv_show has 0 null values
Column frequency has 0 null values
Column to_eat_1 has 0 null values
Column to_eat_2 has 0 null values
Column to_eat_3 has 3 null values
Column to_eat_4 has 7 null values
Column to_eat_5 has 15 null values
Column at_home_1 has 0 null values
Column at_home_2 has 0 null values
Column at_home_3 has 19 null values
Column at_restaurant_1 has 0 null values
Column at_restaurant_2 has 0 null values
Column at_restaurant_3 has 23 null values
Column cook_prep_1 has 0 null values
Column cook_prep_2 has 0 null values
Column cook_prep_3 has 58 null values
Column fruits_1 has 0 null values
Column fruits_2 has 23 null values
Column fruits_3 has 32 null values


## Should we add "NaN" to the blank cells? I'm not sure how...

In [625]:
# change gender column to a string so we can change the words to number values
survey_df['gender'].astype(str)
survey_df['age'].astype(str)
survey_df['age'].astype(float)
survey_df.dtypes

ValueError: could not convert string to float: '40-49'

In [588]:
# look at gender value counts 
gender_count = survey_df['gender'].value_counts()
gender_count

Female    125
Male       35
Tree        1
Name: gender, dtype: int64

In [589]:
# transform the gender column
def changeStatus(status):
    if status == "Female":
        return 1
    if status == "Male":
        return 0
    else:
        return 3

# Along with replace() and map(), this is another way to encode the gender column into numbers.
survey_df["gender"] = survey_df["gender"].apply(changeStatus)
survey_df

,age,gender,race,region,tv_show,frequency,to_eat_1,to_eat_2,to_eat_3,to_eat_4,...,at_home_3,at_restaurant_1,at_restaurant_2,at_restaurant_3,cook_prep_1,cook_prep_2,cook_prep_3,fruits_1,fruits_2,fruits_3
0,40-49,1,"Caucasian, non-Hispanic;",Midwest,Firefly,1/week,Asparagus,Corn,Carrots,Spinach,...,Spinach,Asparagus,Corn,Broccoli,Grilled,Sauteed,Baked,Cherries,blueberries,OTHER
1,20-29,1,Asian;,Midwest,Vincenzo,4-5/week,Broccoli,Cauliflower,Corn,Green Beans,...,Peas,Broccoli,Cauliflower,Corn,Fried/Air-fried,Grilled,Sauteed,OTHER,None,None
2,40-49,1,"Caucasian, non-Hispanic;",Midwest,Ted Lasso,Daily 6-7 week,Broccoli,Carrots,Green Beans,Spinach,...,Squash,Lettuce (any kind),Broccoli,Spinach,Steamed,Baked,,bananas,strawberries,OTHER
3,20-29,1,Black or African American;,Midwest,Succession,2-3/week,Broccoli,Carrots,Corn,Green Beans,...,Cauliflower,Broccoli,Carrots,Green Beans,Baked,Fried/Air-fried,Steamed,Bananas,strawberries,oranges
4,30-39,1,"Caucasian, non-Hispanic;",Midwest,Critical Role,2-3/week,Carrots,Spinach,Squash,Corn,...,Broccoli,Spinach,Carrots,Squash,Baked,Sauteed,,OTHER,OTHER,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,20-29,1,Asian;,Midwest,90 Day Fiance,Daily 6-7 week,Lettuce (any kind),Spinach,Corn,asparagus,...,Carrots,brussel sprouts,Spinach,Corn,Fried/Air-fried,Sauteed,,blueberries,strawberries,grapes
157,30-39,1,"Caucasian, non-Hispanic;",Midwest,Gilmore girls,Daily 6-7 week,Spinach,Lettuce (any kind),Carrots,Broccoli,...,Lettuce (any kind),Squash,Spinach,Cauliflower,Baked,Sauteed,Steamed,Bananas,apples,grapes
158,40-49,1,"Caucasian, non-Hispanic;",Midwest,Chicago Fire,Daily 6-7 week,Lettuce (any kind),Corn,Cauliflower,Carrots,...,Cauliflower,Asparagus,,None,Fried/Air-fried,Grilled,Steamed,Bananas,grapes,apples
159,30-39,1,"Caucasian, non-Hispanic;",Midwest,Bones,2-3/week,Carrots,Corn,Peas,Spinach,...,Peas,Carrots,Green Beans,Spinach,Steamed,,None,Apples,oranges,pineapple


In [608]:
survey_df_columns = survey_df.columns.values.tolist()
survey_df_columns.remove('gender')
survey_df_columns


['age',
 'race',
 'region',
 'tv_show',
 'frequency',
 'to_eat_1',
 'to_eat_2',
 'to_eat_3',
 'to_eat_4',
 'to_eat_5',
 'at_home_1',
 'at_home_2',
 'at_home_3',
 'at_restaurant_1',
 'at_restaurant_2',
 'at_restaurant_3',
 'cook_prep_1',
 'cook_prep_2',
 'cook_prep_3',
 'fruits_1',
 'fruits_2',
 'fruits_3']

In [624]:
# Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(survey_df[survey_df_columns])

ValueError: could not convert string to float: '40-49'

# Categorizing the TV Shows into genres using TheMovieDB API

In [591]:
# #Dependencies for API
# import requests
# import json

# from api_key import movie_api

In [592]:
# shows = survey_df["tv_show"].tolist()
# print(shows)

In [593]:
# movie_url = "https://api.themoviedb.org/3/search/movie?api_key="+movie_api

# ids = []
# titles = []
# genre_id = []
# char_count = []

# for movie in shows:
#     search = requests.get(movie_url + "&query="+movie).json()
#     # search = requests.get(movie_url + "&query= CRITICAL ROLE").json()
#     try:
#         movie_id = search['results'][0]['id']
#         titles.append(movie)
#         genre_id.append(search['results'][0]['genre_ids'])
#         # cast_url = "https://api.themoviedb.org/3/movie/"+str(movie_id)+"/credits?api_key="+movie_api
#         # response = requests.get(cast_url).json()
    
#         # char_count.append(len(response['cast']))
    
#         #Pulls character list for each movie and adds the movie id into the 'ids' list
#         # for i in range(0,(len(response['cast']))):

#         #     chars.append(response['cast'][i]['character'])
#         #     ids.append(movie_id)
#         #     titles.append(movie)
#         # print(movie)
        
            
#     except:
#         print(f"'{movie}' failed to print")
